[KoGPT2훈련](https://github.com/nawnoes/WellnessConversation-LanguageModel/blob/master/train/train-kogpt2-for-wellness.ipynb)

- 데이터먼저 이거에 맞게 다듬기

# 설치

In [1]:
!git clone https://github.com/Jang-jiwon/ChatAi.git

Cloning into 'ChatAi'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [2]:
%cd /content/ChatAi
!git submodule update --init

/content/ChatAi
Submodule 'WellnessConversation-LanguageModel' (https://github.com/nawnoes/WellnessConversation-LanguageModel.git) registered for path 'WellnessConversation-LanguageModel'
Cloning into '/content/ChatAi/WellnessConversation-LanguageModel'...
Submodule path 'WellnessConversation-LanguageModel': checked out 'ad2b3388334c222a48c9c50d0e57890ff487522d'


In [1]:
!git clone https://github.com/nawnoes/WellnessConversation-LanguageModel.git

Cloning into 'WellnessConversation-LanguageModel'...
remote: Enumerating objects: 251, done.
remote: Total 251 (delta 0), reused 0 (delta 0), pack-reused 251
Receiving objects: 100% (251/251), 78.90 KiB | 15.78 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [2]:
cd /content/WellnessConversation-LanguageModel

/content/WellnessConversation-LanguageModel


In [ ]:
!pip install -r requirements.txt

In [4]:
!nvidia-smi

Sun Sep  4 02:40:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import dataloader
from dataloader.wellness import WellnessAutoRegressiveDataset
from model.kogpt2 import DialogKoGPT2

In [6]:
torch.cuda.is_available()

True

# 훈련

In [8]:
mkdir checkpoint

In [9]:
#https://drive.google.com/file/d/13kLUZbtl-CCTS3w1U_Vfqzrq8mYbL2cP/view?usp=sharing
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=13kLUZbtl-CCTS3w1U_Vfqzrq8mYbL2cP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=13kLUZbtl-CCTS3w1U_Vfqzrq8mYbL2cP" -O kogpt2-wellnesee-auto-regressive2.pth && rm -rf ~/cookies.txt

--2022-09-04 02:43:04--  https://docs.google.com/uc?export=download&confirm=t&id=13kLUZbtl-CCTS3w1U_Vfqzrq8mYbL2cP
Resolving docs.google.com (docs.google.com)... 74.125.24.138, 74.125.24.113, 74.125.24.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jmfqfogtsjuprnl6up47157triknth4t/1662259350000/01133989648070413209/*/13kLUZbtl-CCTS3w1U_Vfqzrq8mYbL2cP?e=download&uuid=92c59207-6a9a-4f66-8c71-bd2f2aa272be [following]
--2022-09-04 02:43:04--  https://doc-0c-bo-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jmfqfogtsjuprnl6up47157triknth4t/1662259350000/01133989648070413209/*/13kLUZbtl-CCTS3w1U_Vfqzrq8mYbL2cP?e=download&uuid=92c59207-6a9a-4f66-8c71-bd2f2aa272be
Resolving doc-0c-bo-docs.googleusercontent.com (doc-0c-bo-docs.googleusercontent.com)... 172.217.194.132, 240

In [11]:
root_path='/content/WellnessConversation-LanguageModel'
data_path = f"{root_path}/trainTxt.txt"
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive.pth"

n_epoch = 3         # Num of Epoch
batch_size = 2      # 배치 사이즈
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)
save_step = 100 # 학습 저장 주기
learning_rate = 5e-5  # Learning Rate

dataset= WellnessAutoRegressiveDataset(data_path)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

checkpoint = torch.load('/content/WellnessConversation-LanguageModel/kogpt2-wellnesee-auto-regressive2.pth', map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

#model = DialogKoGPT2()
model.to(device)


loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses =[]
for epoch in range(n_epoch):
    count = 0
    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = torch.stack(data)  # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
            data = data.transpose(1, 0)
            data= data.to(ctx)

            outputs = model(data, labels=data)
            _, logits = outputs[:2]

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = data[..., 1:].contiguous()

            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            # if count % 10 == 0:
            #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
            if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                torch.save({
                    'epoch': epoch,
                    'train_no': count,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss
                }, save_ckpt_path)
            count += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Train(2): 100%|██████████| 3570/3570 [42:13<00:00,  1.41it/s, Loss: 0.376 (0.382)]


새벽6시쯤

In [18]:
import os
import numpy as np
import torch
from model.kogpt2 import DialogKoGPT2
from kogpt2_transformers import get_kogpt2_tokenizer

root_path = '/content/WellnessConversation-LanguageModel'
data_path = f"/content/WellnessConversation-LanguageModel/trainTxt.txt"
checkpoint_path =f"{root_path}/checkpoint"
# save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive.pth"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load('/content/WellnessConversation-LanguageModel/checkpoint/kogpt2-wellnesee-auto-regressive.pth', map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

while 1:
# for i in range(5):
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)
  # set top_k to 50
  sample_output = model.generate(input_ids=input_ids)


  print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')

# for s in kss.split_sentences(sent):
#     print(s)

Question: 요즘 우울한 기분이야


Answer:  무슨 일이 있나요? 무슨 일이 있나요? 무슨 일이 있나요? 무슨 일이 있나요? 무슨 일이 있나요? 무슨 일이 있나요? 무슨 일이 있나요? 무슨 일이 있나요? 있나요?
----------------------------------------------------------------------------------------------------
Question: 우울해


Answer:  우울해. 마음이 불안하시군요. 무슨 일이 있나요?무슨 일이 있나요?불안함을 해소할 방법이 있나요?무슨 일이 있나요?무슨 일이 있나요?무슨 일이 있나요?무슨 일이 있나요?무슨
----------------------------------------------------------------------------------------------------
Question: 친구랑 서먹해


Answer: 서먹한 친구가 있다니 속상하시겠어요.서먹한 친구를 어떻게 하면 변화시킬 수 있을까요?서먹한 친구를 변화시킬 수 있는 방법이 있나요?서먹한 친구를 어떻게 하면 변화시킬 수 있을까요?
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True)

'서먹한 친구가 있다니 속상하시겠어요.서먹한 친구를 어떻게 하면 변화시킬 수 있을까요?서먹한 친구를 변화시킬 수 있는 방법이 있나요?서먹한 친구를 어떻게 하면 변화시킬 수 있을까요?'

15